In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing import *

2023-04-13 13:00:45.330233: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 13:00:45.451881: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [3]:
# SwiCh Activation Function
def swiCh(x):
    return tf.where(x>=0, x, x * (tf.sigmoid(x) * (tf.exp(x) + 1)))


# SwiCh Layer
class SwiCh(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SwiCh, self).__init__(**kwargs)

    def call(self, inputs):
        return swiCh(inputs)

In [4]:
def dense_block(x, blocks, growth_rate):
    for i in range(blocks):
        x = conv_block(x, growth_rate)
    return x

def conv_block(x, growth_rate):
    x1 = BatchNormalization()(x)
    
    
    
    x1 = Activation(swiCh)(x1)
    
    
    
    x1 = Conv2D(4 * growth_rate, (1, 1), use_bias=False, padding='same')(x1)
    x1 = BatchNormalization()(x1)
    
    
    
    x1 = Activation(swiCh)(x1)
    
    
    
    x1 = Conv2D(growth_rate, (3, 3), use_bias=False, padding='same')(x1)
    x = tf.keras.layers.Concatenate()([x, x1])
    return x

def transition_block(x, reduction):
    x = BatchNormalization()(x)
    
    
    
    x = Activation(swiCh)(x)
    
    
    
    x = Conv2D(int(tf.keras.backend.int_shape(x)[3] * reduction), (1, 1), use_bias=False, padding='same')(x)
    x = AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x

def densenet(blocks=[6, 12, 64, 48], growth_rate=32, include_top=True, weights=None, input_shape=None, classes=1000):
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(inputs)
    x = Conv2D(64, (7, 7), strides=(2, 2), use_bias=False)(x)
    x = BatchNormalization()(x)
    
    
    
    x = Activation(swiCh)(x)
    
    
    
    x = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    for i, num_blocks in enumerate(blocks):
        x = dense_block(x, num_blocks, growth_rate)
        if i != len(blocks) - 1:
            x = transition_block(x, 0.5)

    if include_top:
        x = GlobalAveragePooling2D()(x)
        x = Dense(classes, activation='softmax')(x)
        
    model = Model(inputs, x, name='densenet')
    return model

model = densenet(input_shape=(32,32,3), classes=10)
model.summary()

2023-04-13 13:00:47.774884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 13:00:47.778292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 13:00:47.778505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 13:00:47.778893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "densenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization (BatchNorm  (None, 16, 16, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                

In [6]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)


# Define data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
)


In [7]:
wandb.init(project="CIFAR", entity="hcim", name='DenseNet_SwiCh')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=[CategoricalAccuracy()])

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=100, 
                    validation_data=(x_test, y_test), callbacks=[WandbCallback()])

Epoch 1/100


2023-04-13 13:01:46.262951: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/densenet/activation_263/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-04-13 13:01:52.205854: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


782/782 [==============================] - 127s 120ms/step - loss: 1.5354 - categorical_accuracy: 0.4715 - val_loss: 2.2577 - val_categorical_accuracy: 0.3678
Epoch 2/100
782/782 [==============================] - 88s 112ms/step - loss: 1.0940 - categorical_accuracy: 0.6152 - val_loss: 1.4310 - val_categorical_accuracy: 0.5312
Epoch 3/100
782/782 [==============================] - 88s 112ms/step - loss: 0.9371 - categorical_accuracy: 0.6734 - val_loss: 1.0088 - val_categorical_accuracy: 0.6499
Epoch 4/100
782/782 [==============================] - 88s 112ms/step - loss: 0.8322 - categorical_accuracy: 0.7111 - val_loss: 0.8919 - val_categorical_accuracy: 0.6950
Epoch 5/100
782/782 [==============================] - 88s 112ms/step - loss: 0.7510 - categorical_accuracy: 0.7385 - val_loss: 1.2480 - val_categorical_accuracy: 0.6134
Epoch 6/100
782/782 [==============================] - 88s 112ms/step - loss: 0.6846 - categorical_accuracy: 0.7638 - val_loss: 0.7834 - val_categorical_accuracy